In [1]:
from pathlib import Path


# data manager and analysis
import vodex as vx
import numan as nu


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-k81x38rv because the default path (/home/ply/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# DONT TRUST. IT NEEDS TO BE UPDATED. Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
...........................................................
....................... DONE in 01 ........................
...........................................................
processed                                               ...
│   experiment.json <-----------------------------------... the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------... everything about the experiment, but loads from the dff movie, not from the raw data                   ...
└───dff_movie  <----------------------------------------...the dff movie :)
│   │   dff_movie_0000.tif                              ...
│   │   dff_movie_0001.tif                              ...
│   │   ...                                             ...
│..........................................................
│...................... DONE in 02 ........................
│..........................................................
│                                                       ...
└───tscore_volumes  <-----------------------------------... t-score tif files
│   │   tscore_SvB.tif <--------------------------------... t-score Stimuli vs Blank
│   │                                                   ...
│..........................................................
│...................... DONE : MANUAL .....................
│..........................................................
└───spots                                               ...
│   └───imaris  <---------------------------- ATTENTION ... You need to put stuff generated by imaris into this folder!!!                                         ...
│       │   └───tscore_SvB_Statistics                   ...
│       │       │     tscore_SvB_Position.csv           ...
│       │       │     tscore_SvB_Diameter.csv           ...
│       │       │     ...                               ...
│..........................................................
│................... DONE in 03 & 04 ......................
│..........................................................
│   │                                                   ...
│   └───signals  <--------------------------------------... json files with the extracted signals, also will have the group info after you added it                 ...
│       │   spots_SvB_max.json                          ...
│       │                                               ...
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++ WILL BE DONE in this notebook +++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│   │                                                   +++
│   └───reports  <------------------------------------- +++ tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <------------------------- +++ tiffs and pdf with all significant in any way cells
│           │   └───signals  <------------------------- +++ pdfs with signals
│           │       │     ...                           +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│           │       │
│           │   └───images <--------------------------- tif masks
│           │       │     ...
│       │
│       └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images  <----------------------- tif masks
│           │       │     ...
```

# Set project folder

The processed/spots/signals should already exist and have the extracted signals saved in there. 

In [2]:
project_folder = "/home/ply/repos/numan/notebooks/vodex_si/data/"
project = nu.Project(project_folder)

project.check_exists("processed/spots/signals")

project.activate("processed")

# Load experiment with the raw data and define conditions: 

In [3]:
experiment = vx.Experiment.load('experiment_raw.db')

## Create lots of pdfs with lots of traces :

Since we have so much different ways that a cell can be significant, I think I'll just out put all the cells that are significant in any way at all .. and will indicate the way that they are significant ... Not the best way of doing things in general.. but oh well...

## Create a folder and initialise the report maker

In [5]:
project.create("processed/spots/reports/all_significant/signals")
report = nu.Reports(Path(project.main_folder,"processed"), experiment)

## Make cycle plots: 
Without individual traces

In [7]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            # types of plots:
                            plot_type = "cycle",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '',
                            # forward_shift will shift the cycle by the set number of voxels
                            # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                            # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                            forward_shift=3,
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF


With individual traces

In [8]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            # types of plots:
                            plot_type = "cycle",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_individ_traces',
                            # forward_shift will shift the cycle by the set number of voxels
                            # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                            # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                            forward_shift=3,
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF


## Make PSH plots: 
### Only stimulus 
List the cycle timepoints that you want to show and ovrelap ...

In [9]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF


In [10]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # types of plots:
                            plot_type = "psh_0",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF


### Stimulus and 2 blanks before and 4 after

In [11]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '',
                            # whether to plot the individual traces
                            plot_individual=False)

Using sliding window 15 volumes for signal DFF


In [12]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True)

Using sliding window 15 volumes for signal DFF


## Scatterplots with average activity per stimuli:

In [13]:
report.make_avg_intensity_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # whether to number the cells in the plots:
                            number_cells = False,
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '')

Using sliding window 15 volumes for signal DFF


In [14]:
report.make_avg_intensity_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # whether to number the cells in the plots:
                            number_cells = True,
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type, but something different
                            plot_type_tag = '_numbered_cells')

Using sliding window 15 volumes for signal DFF


### Prepare pdfs with checkboxes for groupping
The following code will create a folder "processed/spots/reports/groupped/signals" and generate there the same plots as the ones above, but with checkboxes to select cells. The image quality is worth there , so for presentations use the previous plots.

In [15]:
project.create('processed/spots/reports/groupped/signals')

In [16]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            # types of plots:
                            plot_type = "cycle",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '',
                            # forward_shift will shift the cycle by the set number of voxels
                            # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                            # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                            forward_shift=3,
                            # whether to plot the individual traces
                            plot_individual=False,
                            checkbox=True)

Using sliding window 15 volumes for signal DFF


In [17]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            # types of plots:
                            plot_type = "cycle",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_individ_traces',
                            # forward_shift will shift the cycle by the set number of voxels
                            # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                            # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                            forward_shift=3,
                            # whether to plot the individual traces
                            plot_individual=True,
                            checkbox=True)

Using sliding window 15 volumes for signal DFF


In [18]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d4","d5"],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '',
                            # whether to plot the individual traces
                            plot_individual=False,
                            checkbox=True)

Using sliding window 15 volumes for signal DFF


In [19]:
report.make_signal_reports("SvB_max", "sigAny1v2v3v4v5vB",
                           "number",
                            labels= ["d1","d2","d3","d5"],
                            # types of plots:
                            plot_type = "psh_b",
                            # which groups to write above the plots, also sorts but how many of these are significant
                            groups_to_specify=("sig1v2","sig1v3","sig1v4","sig1v5",
                                               "sig2v3","sig2v4", "sig2v5", 
                                               "sig3v4","sig3v5",
                                               "sig4v5",
                                               "sig1vB","sig2vB", "sig3vB","sig4vB", "sig5vB"),
                            # just for the pdf naming :
                            # this is to be able to distinguish the pdfs with the same plot type,
                            # but errors are different or raw traces on/off or front_to_tail
                            plot_type_tag = '_individ_traces',
                            # whether to plot the individual traces
                            plot_individual=True,
                            checkbox=True)

Using sliding window 15 volumes for signal DFF
